# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [10]:
# import libraries
import numpy as np
import pandas as pd
import re
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report,fbeta_score,make_scorer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import pickle
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shenhaichen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shenhaichen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shenhaichen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///disater_response.db')
df = pd.read_sql_table('disater_response', engine)
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [3]:
X = df['message']
Y = df.drop(columns = ['id','message','original','genre'], axis = 1)

In [4]:
column_names= Y.columns
column_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [23]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [25]:
np.random.seed(42)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [26]:
y_pred = pipeline.predict(X_test)

In [27]:
print(classification_report(y_test, y_pred, target_names=column_names))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      3998
               request       0.83      0.34      0.48       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.77      0.52      0.62      2164
          medical_help       0.56      0.06      0.11       435
      medical_products       0.76      0.10      0.18       279
     search_and_rescue       0.50      0.06      0.11       136
              security       0.17      0.01      0.02        96
              military       0.53      0.06      0.10       158
           child_alone       0.00      0.00      0.00         0
                 water       0.82      0.28      0.42       335
                  food       0.86      0.32      0.47       584
               shelter       0.79      0.24      0.37       468
              clothing       1.00      0.03      0.06        70
                 money       0.60      

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
# check parameter then decide which parameters have to be optimized 
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f4913aa4400>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [29]:
parameters = { 'clf__estimator__min_samples_split': [2,4],
               'clf__estimator__n_estimators': [10,100,150],
               'clf__estimator__random_state': [None, 42],
               'tfidf__use_idf':[True, False],
                }

In [30]:
cv = GridSearchCV(pipeline, param_grid = parameters, verbose=10, cv=5)

In [31]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=None, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=None, tfidf__use_idf=True, score=0.22479141835518474, total=  42.6s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=None, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   52.8s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=None, tfidf__use_idf=True, score=0.22216924910607866, total=  42.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=None, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=None, tfidf__use_idf=True, score=0.2312827849308536, total=  42.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=None, tfidf__use_idf=True 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=None, tfidf__use_idf=False, score=0.23104434907010013, total=  41.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=None, tfidf__use_idf=False 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  7.8min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=None, tfidf__use_idf=False, score=0.22937529804482595, total=  40.6s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=42, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=42, tfidf__use_idf=True, score=0.22169249106078665, total=  40.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=42, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=42, tfidf__use_idf=True, score=0.22073897497020262, total=  40.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__random_state=42, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__rando

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 587.2min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__min_samples_split': [2, 4], 'clf__estimator__n_estimators': [10, 100, 150], 'clf__estimator__random_state': [None, 42], 'tfidf__use_idf': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [32]:
y_pred_optimized = cv.predict(X_test)
print(classification_report(y_test, y_pred_optimized, target_names=column_names))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      3998
               request       0.89      0.43      0.58       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.80      0.60      0.69      2164
          medical_help       0.62      0.04      0.07       435
      medical_products       0.71      0.04      0.08       279
     search_and_rescue       0.50      0.04      0.07       136
              security       0.33      0.01      0.02        96
              military       0.82      0.06      0.11       158
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.21      0.34       335
                  food       0.90      0.38      0.53       584
               shelter       0.88      0.25      0.39       468
              clothing       1.00      0.06      0.11        70
                 money       0.89      

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


As we can see, all score has increase, which means the Grid Search totally improve the model, however, they just raise few score point. I think the algorithm is not quite suit for it. I would like to try another algorithm.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [33]:
# check the best parameter for in case
cv.best_params_

{'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 150,
 'clf__estimator__random_state': None,
 'tfidf__use_idf': True}

In [8]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [15]:
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer(use_idf=True))
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=None, n_estimators=150)))
])


In [ ]:
# parameters = {
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 5000),
#     }

In [16]:
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...ator=None,
          learning_rate=1.0, n_estimators=150, random_state=None),
           n_jobs=1))])

In [17]:
y_pred2 = pipeline2.predict(X_test)

In [18]:
print(classification_report(y_test, y_pred2, target_names=column_names))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      3998
               request       0.77      0.56      0.65       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.76      0.63      0.69      2164
          medical_help       0.60      0.32      0.42       435
      medical_products       0.55      0.28      0.37       279
     search_and_rescue       0.42      0.16      0.23       136
              security       0.15      0.03      0.05        96
              military       0.56      0.34      0.42       158
           child_alone       0.00      0.00      0.00         0
                 water       0.70      0.63      0.66       335
                  food       0.80      0.70      0.75       584
               shelter       0.73      0.54      0.63       468
              clothing       0.64      0.41      0.50        70
                 money       0.50      

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


The score of precision is quite close to the previous model, but another score is extremely higher than the previous one. If I imporve the model, I think it would be better than current model.

### 9. Export your model as a pickle file

In [19]:
# save the model to disk
filename = 'classifier.sav'
pickle.dump(pipeline2, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.